# Bias Evaluation : AIF360
***

Quantification of model bias in terms of fairness against protected groups before and after implementation of mitigation methods

## Terminology
***

***Favorable label:*** A label whose value corresponds to an outcome that provides an advantage to the recipient (such as receiving a loan, being hired for a job, not being arrested)

***Protected attribute:*** An attribute that partitions a population into groups whose outcomes should have parity (such as race, gender, caste, and religion)

***Privileged value (of a protected attribute):*** A protected attribute value indicating a group that has historically been at a systemic advantage

***Fairness metric:*** A quantification of unwanted bias in training data or models

***Discrimination/unwanted bias:*** Although bias can refer to any form of preference, fair or unfair, our focus is on undesirable bias or discrimination, which is when specific privileged groups are placed at a systematic advantage and specific unprivileged groups are placed at a systematic disadvantage. This relates to attributes such as race, gender, age, and sexual orientation.


## Structure of Evaluation & Intervention
***
<img src="images/aif360_pipeline.png" width="700" height="500" align="center"/>

### Three Perspectives of Fairness in ML algorithms
***

[linkedin article](https://www.linkedin.com/pulse/whats-new-deep-learning-research-reducing-bias-models-jesus-rodriguez/)

***1. Data vs Mode***

Fairness may be quantified in the training dataset or in the learned model

***2. Group vs Individual***

Group fairness partitions a population into groups defined by protected attributes and seeks for some statistical measure to be equal across all groups. Individual fairness seeks for similar individuals to be treated similarly.


***3. WAE vs WYSIWYG (We are all equal vs What you see is what you get)***

WAE says that fairness is an equal distirbution of skills and opportunities among the participants in an ML task, attributing differences in outcome distributions to structural bias and not a difference in distribution to ability. WYSIWYG says that observations reflect ability with respect to a task.

> If the application follows the WAE worldview, then the demographic parity metrics should be used: disparate_impact and statistical_parity_difference.  If the application follows the WYSIWYG worldview, then the equality of odds metrics should be used: average_odds_difference and average_abs_odds_difference.  Other group fairness metrics (some are often labeled equality of opportunity) lie in-between the two worldviews and may be used appropriately: false_negative_rate_ratio, false_negative_rate_difference, false_positive_rate_ratio, false_positive_rate_difference, false_discovery_rate_ratio, false_discovery_rate_difference, false_omission_rate_ratio, false_omission_rate_difference, error_rate_ratio, and error_rate_difference. 

### 2. Evaluation of Bias in Models

<center><b>Average Odds Difference:</b> $ \tfrac{1}{2}\left[(FPR_{D = \text{unprivileged}} - FPR_{D = \text{privileged}}) + (TPR_{D = \text{privileged}} - TPR_{D = \text{unprivileged}}))\right] $ </center>


<br><br>


<center><b>Statistical Parity Difference:    </b>$ Pr(\hat{Y} = 1 | D = \text{unprivileged}) - Pr(\hat{Y} = 1 | D = \text{privileged}) $</center>


<br><br>


<center><b>Equal Opportunity Difference:    </b>$ TPR_{D = \text{unprivileged}} - TPR_{D = \text{privileged}} $</center>


<br><br>


<center><b>Theil Index:    </b>$ \frac{1}{n}\sum_{i=1}^n\frac{b_{i}}{\mu}\ln\frac{b_{i}}{\mu}, \text{with} b_i = \hat{y}_i - y_i + 1 $</center>


<br><br>


<center><b>Disparate Impact:    </b> $ \frac{Pr(Y = 1 | D = \text{unprivileged})} {Pr(Y = 1 | D = \text{privileged})}$</center>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(context='talk', style='whitegrid')
from IPython.display import display, Markdown

from tensorflow import keras
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.optimizers  import Adam, Adagrad, SGD, RMSprop

# from aif360.sklearn.metrics import mdss_bias_scan, mdss_bias_score
import aif360
import utilities
import global_variables as gv
import fairness_helpers as fh

In [2]:
from aif360.datasets import StandardDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.sklearn import metrics as mt
from aif360.explainers import MetricTextExplainer, MetricJSONExplainer

### load data

In [3]:
X, X1 = fh.get_aif360_data()
X

1319-0.0  1408-0.0  1329-0.0  \
sex-binary race-binary race-grouped age-binary                                 
0          1           0            1                0.0       1.0       2.0   
1          1           0            1                0.0       3.0       2.0   
0          1           0            1                0.0       3.0       3.0   
1          1           0            1                3.0       3.0       3.0   
                                    0                0.0       3.0       2.0   
...                                                  ...       ...       ...   
                                    1                0.0       3.0       1.0   
0          1           0            1                0.0       1.0       1.0   
1          1           0            0                2.0       1.0       1.0   
0          1           0            1                3.0       3.0       1.0   
1          1           0            1                1.0       2.0       2.0   

                                                1448-0.0  1538-0.0  6142-0.0  \
sex-binary race-binary race-grouped age-binary                                 
0          1           0            1                3.0       2.0       1.0   
1          1           0            1                1.0       0.0       1.0   
0          1           0            1                2.0       1.0       2.0   
1          1           0            1                3.0       0.0       2.0   
                                    0                1.0       0.0       5.0   
...                                                  ...       ...       ...   
                                    1                1.0       2.0       1.0   
0          1           0            1                3.0       2.0       2.0   
1          1           0            0                2.0       2.0       1.0   
0          1           0            1                3.0       0.0       1.0   
1          1           0            1                1.0       0.0       1.0   

                                                2050-0.0  1508-0.0  1339-0.0  \
sex-binary race-binary race-grouped age-binary                                 
0          1           0            1                2.0       3.0       2.0   
1          1           0            1                1.0       2.0       2.0   
0          1           0            1                1.0       2.0       2.0   
1          1           0            1                1.0       2.0       2.0   
                                    0                2.0       2.0       2.0   
...                                                  ...       ...       ...   
                                    1                1.0       1.0       2.0   
0          1           0            1                1.0       1.0       1.0   
1          1           0            0                4.0       2.0       2.0   
0          1           0            1                1.0       2.0       2.0   
1          1           0            1                1.0       2.0       2.0   

                                                30710-0.0  1349-0.0  \
sex-binary race-binary race-grouped age-binary                        
0          1           0            1              0.3400       1.0   
1          1           0            1              3.9400       4.0   
0          1           0            1              0.5500       1.0   
1          1           0            1              0.4500       2.0   
                                    0              0.7500       2.0   
...                                                   ...       ...   
                                    1              1.4500       3.0   
0          1           0            1              2.7500       1.0   
1          1           0            0              0.8500       3.0   
0          1           0            1              3.5700       2.0   
1          1           0            1              2.9031       1.0   

 

In [4]:
gv.protected_attributes # sex, race, age

['31-0.0', '21000-0.0', '21003-0.0']

### Evaluate bias in original dataset: Disparate impact ratio

#### Interpretation:

<ul>
    <li> output range=[0,1]</li>
    <li> a higher value == more fair related to the given protected attribute</li>
    <li> x>0.8 is considered acceptable bias</li>

In [5]:
mt.disparate_impact_ratio(X['CVD'], prot_attr='sex-binary')

0.5661578163243346

In [6]:
mt.disparate_impact_ratio(X['CVD'], prot_attr='race-binary')

0.5438220418530659

In [7]:
mt.disparate_impact_ratio(X['CVD'], prot_attr='age-binary')

0.31036008621940697

> disparate impact ratios indicate that all three investigated protected attributes possess significant bias

### load model

In [9]:
model = keras.models.load_model('saved_models/mlp_binary_1.h5')
model.compile(loss='categorical_hinge',
              optimizer=SGD(learning_rate=0.0005),
              metrics=['acc',tf.keras.metrics.AUC(), tf.keras.metrics.Recall()])

#### get predictions

In [10]:
X_inputs = X1.drop(fh.protected_attribute_names+['CVD'], axis=1)
y = X1['CVD']
X_train, X_test, y_train, y_test = train_test_split(X1.drop('CVD', axis=1), y, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

In [11]:
X_train

,1319-0.0,1408-0.0,1329-0.0,2050-0.0,1339-0.0,30710-0.0,1349-0.0,30750-0.0,30740-0.0,1160-0.0,1488-0.0,30850-0.0,4080-0.0,1369-0.0,1200-0.0,1289-0.0,30790-0.0,845-0.0,48-0.0,30630-0.0,1299-0.0,1220-0.0,1548-0.0,1528-0.0,23099-0.0,49-0.0,30690-0.0,1389-0.0,1249-0.0,1309-0.0,1379-0.0,21003-0.0,30780-0.0,1438-0.0,30870-0.0,1359-0.0,30770-0.0,21001-0.0,1458-0.0,23100-0.0,1418-0.0,1478-0.0,4079-0.0,30760-0.0,23101-0.0,30640-0.0,1428-0.0_1.0,1428-0.0_2.0,1428-0.0_3.0,20117-0.0_1.0,20117-0.0_2.0,2100-0.0_1.0,2654-0.0_4.0,2654-0.0_5.0,2654-0.0_6.0,2654-0.0_7.0,2654-0.0_8.0,2654-0.0_9.0,21000-0.0_2.0,21000-0.0_3.0,21000-0.0_4.0,21000-0.0_5.0,21000-0.0_6.0,21000-0.0_1001.0,21000-0.0_1002.0,21000-0.0_1003.0,21000-0.0_2001.0,21000-0.0_2002.0,21000-0.0_2003.0,21000-0.0_2004.0,21000-0.0_3001.0,21000-0.0_3002.0,21000-0.0_3003.0,21000-0.0_3004.0,21000-0.0_4001.0,21000-0.0_4002.0,21000-0.0_4003.0,1538-0.0_1.0,1538-0.0_2.0,31-0.0_1.0,6138-0.0_2.0,6138-0.0_3.0,6138-0.0_4.0,6138-0.0_5.0,6138-0.0_6.0,2090-0.0_1.0,1508-0.0_2.0,1508-0.0_3.0,1508-0.0_4.0,6142-0.0_2.0,6142-0.0_3.0,6142-0.0_4.0,6142-0.0_5.0,6142-0.0_6.0,6142-0.0_7.0,1468-0.0_2.0,1468-0.0_3.0,1468-0.0_4.0,1468-0.0_5.0,1239-0.0_1.0,1239-0.0_2.0,1448-0.0_2.0,1448-0.0_3.0,1448-0.0_4.0,hypertension_1,sex-binary,race-binary,race-grouped,age-binary
371350,1.82,2.0,2.0,1.0,2.0,-0.532102,3.0,-0.857143,3.487884,0.0,0.666667,-0.079326,-0.1208,2.0,1.0,-0.14,-0.396939,0.422727,-0.855615,0.382114,-0.5,0.0,3.0,0.000000,0.388889,-0.454545,0.285714,1.0,2.0,0.5,1.0,0.076923,-0.017560,0.666667,-0.737736,2.0,0.148344,-0.489359,0.120098,-0.061947,2.0,1.0,-0.350000,1.317582,-0.560847,-0.267797,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1
209258,0.00,2.0,1.0,1.0,2.0,11.639045,2.0,0.224490,-0.458805,-1.0,0.333333,1.081857,0.5600,2.0,3.0,-0.14,-0.627745,0.000000,0.534759,-0.647696,-0.5,0.0,3.0,-0.333333,-0.150794,0.181818,-0.823162,1.0,1.0,-0.5,1.0,0.076923,-0.608133,-0.222222,-0.359434,3.0,0.708756,0.396322,0.000000,0.274336,2.0,2.0,0.928571,-0.498229,0.719577,-0.677966,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,1,0,1
182890,2.00,3.0,2.0,1.0,1.0,1.951515,2.0,-0.081633,-0.434572,-2.0,2.333333,-0.058692,-0.3600,2.0,2.0,1.86,-0.351460,-0.681818,-0.053476,0.360434,0.0,1.0,1.0,0.000000,0.809524,0.727273,1.609570,1.0,4.0,0.0,1.0,-0.076923,1.597967,0.444444,0.042453,2.0,1.257542,0.502961,-0.245098,0.663717,6.0,2.0,0.071429,0.155697,-0.359788,1.352542,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1
272132,1.00,5.0,3.0,1.0,1.0,4.625083,3.0,0.612245,1.095315,-1.0,1.000000,0.873175,0.2000,1.0,3.0,1.86,-0.199787,0.609091,0.106952,0.482385,-0.5,0.0,1.0,-0.333333,-1.182540,-0.636364,-0.512483,1.0,3.0,-0.5,1.0,0.153846,-0.673752,2.777778,-0.427358,3.0,0.181898,-0.520899,1.225490,-1.044248,2.0,1.0,0.500000,0.264684,0.502646,-0.616949,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1
23289,0.00,3.0,2.0,1.0,1.0,1.049245,3.0,1.183673,2.449111,-1.0,-1.000000,1.157290,0.0800,2.0,2.0,-2.14,0.744192,-0.909091,-0.053476,-0.411924,-1.0,0.0,1.0,-0.666667,-0.023810,-0.727273,-1.027046,2.0,1.0,0.0,3.0,0.000000,-1.073013,0.831111,0.151887,1.0,0.713319,0.028266,-0.818627,-0.150442,6.0,1.0,-1.642857,-0.315286,-0.100529,-0.664407,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [15]:
y_prob_val = model.predict(X_val.drop(fh.protected_attribute_names, axis=1))
y_prob_test = model.predict(X_test.drop(fh.protected_attribute_names, axis=1))

3141/3141 [==============================] - 29s 9ms/step


## True Positive Rate (TPR) Disparity

We quantify the TPR disparity per subgroup/disease for sex and age. Instances of positive and negative disparities, which can denote bias for or against a subgroup indicates *favorable* or *unfavorable* subgroups.

In [ ]:
def tpr_disparity(y_prob, y_true, groups):
    return tpr_disparity

## TPR Disparity in proportion to membership

Investigate the distribution of the positive patient proportion per subgroup and the effect on TPR disparitites

### Protected Attribute: Sex

In [16]:
threshold = 0.8

In [18]:
result = fh.get_fairness(X, threshold, 'sex-binary')
result

3141/3141 [==============================] - 28s 9ms/step


TypeError: '>' not supported between instances of 'numpy.ndarray' and 'str'

In [25]:
result = fh.get_fairness(X, threshold, 'race-binary')
result

3141/3141 [==============================] - 20s 6ms/step


,race-binary
average_odds_difference,-0.016038
disparate_impact_ratio,0.395272
equal_opportunity_difference,-0.027353
statistical_parity_difference,-0.007668


In [26]:
result = fh.get_fairness(X, threshold,'age-binary')
result

3141/3141 [==============================] - 20s 6ms/step


,age-binary
average_odds_difference,-0.021265
disparate_impact_ratio,0.103662
equal_opportunity_difference,-0.032315
statistical_parity_difference,-0.014005


In [ ]:
dataset1 = StandardDataset(X1, 
                          label_name='CVD', 
                          favorable_classes=[1], 
                          protected_attribute_names=['sex-binary'], 
                          privileged_classes=[[1]])
dataset2 = StandardDataset(X1, 
                          label_name='CVD', 
                          favorable_classes=[1], 
                          protected_attribute_names=['race-binary'], 
                          privileged_classes=[[1]])
dataset3 = StandardDataset(X1, 
                          label_name='CVD', 
                          favorable_classes=[1], 
                          protected_attribute_names=['age-binary'], 
                          privileged_classes=[[1]])

In [ ]:
dataset1b = BinaryLabelDataset(df=X1, 
                          label_names=['CVD'], 
                          protected_attribute_names=['sex-binary'])

dataset2b = BinaryLabelDataset(df=X1, 
                          label_names=['CVD'], 
                          protected_attribute_names=['race-binary'])

dataset3b = BinaryLabelDataset(df=X1, 
                          label_names=['CVD'], 
                          protected_attribute_names=['age-binary'])

#### Sex

#### metrics for original training data

In [ ]:
privileged_groups, unprivileged_groups = fh.get_att_privilege_groups('sex-binary')

In [ ]:
# Get the dataset and split into train and test
dataset_orig_train, dataset_orig_vt = dataset1.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

# Get the dataset and split into train and test
dataset_orig_train1b, dataset_orig_vt1b = dataset1b.split([0.7], shuffle=True)
dataset_orig_valid1b, dataset_orig_test1b = dataset_orig_vt1b.split([0.5], shuffle=True)

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

metric_orig_valid = BinaryLabelDatasetMetric(dataset_orig_valid, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original validation dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_valid.mean_difference())

metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

In [ ]:
print('1-min(DI, 1/DI):', fh.get_disparity_index(metric_orig_train.disparate_impact()).round(3))

#### Race

In [ ]:
privileged_groups2, unprivileged_groups2 = fh.get_att_privilege_groups('race-binary')

# Get the dataset and split into train and test
dataset_orig_train2, dataset_orig_vt2 = dataset2.split([0.7], shuffle=True)
dataset_orig_valid2, dataset_orig_test2 = dataset_orig_vt2.split([0.5], shuffle=True)

In [ ]:
metric_orig_train2 = BinaryLabelDatasetMetric(dataset_orig_train2, 
                             unprivileged_groups=unprivileged_groups2,
                             privileged_groups=privileged_groups2)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train2.mean_difference())

metric_orig_valid2 = BinaryLabelDatasetMetric(dataset_orig_valid2, 
                             unprivileged_groups=unprivileged_groups2,
                             privileged_groups=privileged_groups2)
display(Markdown("#### Original validation dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_valid2.mean_difference())

metric_orig_test2 = BinaryLabelDatasetMetric(dataset_orig_test2, 
                             unprivileged_groups=unprivileged_groups2,
                             privileged_groups=privileged_groups2)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test2.mean_difference())

In [ ]:
print('1-min(DI, 1/DI):', fh.get_disparity_index(metric_orig_train2.disparate_impact()).round(3))

#### Age

In [ ]:
privileged_groups3, unprivileged_groups3 = fh.get_att_privilege_groups('age-binary')

# Get the dataset and split into train and test
dataset_orig_train3, dataset_orig_vt3 = dataset3.split([0.7], shuffle=True)
dataset_orig_valid3, dataset_orig_test3 = dataset_orig_vt3.split([0.5], shuffle=True)

In [ ]:
metric_orig_train3 = BinaryLabelDatasetMetric(dataset_orig_train3, 
                             unprivileged_groups=unprivileged_groups3,
                             privileged_groups=privileged_groups3)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train3.mean_difference())

metric_orig_valid3 = BinaryLabelDatasetMetric(dataset_orig_valid3, 
                             unprivileged_groups=unprivileged_groups3,
                             privileged_groups=privileged_groups3)
display(Markdown("#### Original validation dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_valid3.mean_difference())

metric_orig_test3 = BinaryLabelDatasetMetric(dataset_orig_test3, 
                             unprivileged_groups=unprivileged_groups3,
                             privileged_groups=privileged_groups3)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test3.mean_difference())

In [ ]:
print('1-min(DI, 1/DI):', fh.get_disparity_index(metric_orig_train3.disparate_impact()).round(3))

In [ ]:
metric_orig_train3.theil_index()